# 1. Prepare libs and data

In [ ]:
%load_ext dotenv
%dotenv ../services/environments/.env.secrets

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
%load_ext snakeviz

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# Import standard libs
import json
import logging
import os
import pickle
import pycocotools
import random
import regex
import sys
import shutil
from collections import OrderedDict
from copy import deepcopy
from datetime import datetime, timezone
from itertools import accumulate
from operator import add
from pathlib import Path
from pprint import pprint
from typing import *
from uuid import uuid4


# import first party libs
import cv2
import joblib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())
from PIL import Image
from skimage.measure import label, regionprops, find_contours
from tqdm.auto import tqdm


# import detectron2 utilities
# (Some basic setup: Setup detectron2 logger)
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode, Boxes
from detectron2.data import transforms as T
from detectron2.data import build_detection_train_loader, build_detection_test_loader, DatasetMapper
from detectron2.data import detection_utils as utils

from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.config.config import CfgNode
from detectron2.engine import hooks

from detectron2.utils import comm
from detectron2.structures import Instances
from detectron2.evaluation import COCOEvaluator, DatasetEvaluator, DatasetEvaluators
from detectron2.modeling import GeneralizedRCNNWithTTA

from detectron2.checkpoint import DetectionCheckpointer

from detectron2.evaluation import verify_results

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", 100)
setup_logger()
cv2.setNumThreads(0)

# Append codebase path
sys.path.append("../src/")

In [ ]:
from vxgioq_cfu_counting_ml.api.data_models import Predictions, Region, ShapeAttributes

from vxgioq_cfu_counting_ml.training.dataset_mapper import CFUDatasetMapper
from vxgioq_cfu_counting_ml.training.data_analysis import get_dataset_stats, check_retest_models, compute_kpis, get_data_compute_predictions_save_zip_upload_images_names_predictions_count, construct_model_info, predict_images
from vxgioq_cfu_counting_ml.training.matched_data import MATCHES
from vxgioq_cfu_counting_ml.training.organize_data import get_paths_test_images, get_paths_train_images, count_files, get_paths_positive_unlabelled_images, via_to_detectron2, custom_train_test_split, generate_train_test, gsk_original_to_detectron2, negative_to_detectron2
from vxgioq_cfu_counting_ml.training.trainer import CFUTrainer

from vxgioq_cfu_counting_ml.utils.azure import (
    delete_object_from_blob,
    download_load_models_kpis,
    download_unzip_clean_zip,
    get_cosmos_mongo_collection,
    get_file_from_blob,
    list_data_on_blob,
    list_models_on_blob,
    upload_file_to_blob,
    download_unzip_clean_data,
    download_unzip_clean_model,
    write_zip_upload_models_kpis,
    zip_upload_data_to_blob,
    zip_upload_file_folder_to_blob,
    zip_upload_model_to_blob,
    zip_upload_predictions_to_blob,
    ACCOUNT_URL,
    CONTAINER,
    DATASET_DETECTRON2_NAME,
    DATASETS_QUANTILES_NAME,
    MODEL_CONFIG_NAME,
    MODELS_KPIS_NAME,
    MODELS_QUANTILES_NAME,
    MODEL_WEIGHTS_NAME,
    PATH_DATA_ON_DISK,
    PATH_MODELS_ON_DISK,
    PATH_PREDICTIONS_ON_DISK
)
from vxgioq_cfu_counting_ml.utils.conf import Backbone, Device, clean_output_dir_add_cfg, delete_files, delete_folders, prepare_cfu_detectron2_config_from_detectron2_model_zoo, prepeare_cfu_detectron_config_from_existing_model
from vxgioq_cfu_counting_ml.utils.prediction_intervals import pad_arrays, scores_to_distributions, generate_pi_dataset, DataType, train_quantile_regressor, HPSpace, verify_quantiles_predictions, BinningOption, DistributionFunctionType, scores_to_compiled_cdfs
from vxgioq_cfu_counting_ml.utils.convert import enhance_detectron2_dataset, extract_challenging_images, load_scan4000_excel, extract_png_from_scan4000_report, extract_metadata_from_scan4000_report, extract_all_from_scan4000_report
from vxgioq_cfu_counting_ml.utils.custom_datasets import func_dataset, mask_2_polygon
from vxgioq_cfu_counting_ml.utils.display import compare_scan4000_and_dl_images, display_batch, display_pi, display_dataset, display_enhanced_dataset, display_features_importance, display_violin, NormalizeOption, save_examples
from vxgioq_cfu_counting_ml.utils.evaluator import CFUEvaluator
from vxgioq_cfu_counting_ml.utils.loader import load_model, get_best_model_name, get_best_model, load_images
from vxgioq_cfu_counting_ml.utils.metrics import mse, mae, mape, calibration
from vxgioq_cfu_counting_ml.utils.predictor import CFUPredictor
from vxgioq_cfu_counting_ml.utils.types import *


## Download models

In [ ]:
# download and load the models kpis
models_kpis = download_load_models_kpis()


In [ ]:
# list and download all models
model_names = [model.name for model in models_kpis.models if model.available]
model_names

for model_blob in model_names:
    if model_blob != MODELS_KPIS_NAME:  # Don't download the KPIs, we will do that afterwards
        download_unzip_clean_model(model_name=model_blob, overwrite=False)


## Download datasets and convert to detectron2 compatible

In [ ]:
gsk_original_to_detectron2(data_name="labelled", overwrite=False)
negative_to_detectron2(data_name="unlabelled", overwrite=False)
via_to_detectron2(data_name="via_challenging_dl", overwrite=False)
via_to_detectron2(data_name="via_challenging_scan4000", overwrite=False)


## Copy and organize the dataset into train and test

In [ ]:
folders_positive = [
    os.path.join(PATH_DATA_ON_DISK, "labelled"),
    os.path.join(PATH_DATA_ON_DISK, "via_challenging_dl"),
    os.path.join(PATH_DATA_ON_DISK, "via_challenging_scan4000"),
]

folders_negative = [
    os.path.join(PATH_DATA_ON_DISK, "unlabelled"),
]

path_organized = generate_train_test(
    folders=folders_positive,
    folders_negative=folders_negative,
    overwrite=True
)


In [ ]:
# remove exceedingly challenging images from train set as they are susceptible to exploding gradients and GPU RAM saturation
challenging_images_names = [
    "via_challenging_dl_image_00101.png",
    "via_challenging_dl_image_00103.png",
    "via_challenging_dl_image_00104.png",
    "via_challenging_dl_image_00115.png",
    "via_challenging_dl_image_00713.png",
    "via_challenging_dl_image_00754.png",
    "via_challenging_dl_image_01391.png",
#     "via_challenging_dl_image_01333.png",
#     "via_challenging_dl_image_01334.png",
]


def remove_from_dataset(path_detectron2_dataset: str, images_to_remove: List[str]) -> None:
    """Remove images from a detectron2 dataset."""
    # load dataset
    detectron2_dataset = joblib.load(path_detectron2_dataset)
    
    # create new dataset and remove images if need be
    new_detectron2_dataset: List[Detectron2DatasetElement] = []
    images_to_delete: List[str] = []
    for i, data_el in enumerate(detectron2_dataset):
        if not any(image_to_remove in data_el.file_name for image_to_remove in images_to_remove):
            new_detectron2_dataset.append(data_el)
        else:
            images_to_delete.append(data_el.file_name)
    
    # overwrite old dataset with new one
    logger.info(f"Overwriting dataset detectron2...")
    joblib.dump(new_detectron2_dataset, path_detectron2_dataset, compress=9, protocol=pickle.HIGHEST_PROTOCOL)
    
    # delete images
    logger.info(f"Deleting {len(images_to_delete)} images: {images_to_delete}....")
    delete_files(images_to_delete)

    
remove_from_dataset(path_detectron2_dataset=os.path.join(path_organized, "train", DATASET_DETECTRON2_NAME), images_to_remove=challenging_images_names)


In [ ]:
sorted(os.listdir(os.path.join(path_organized, "test")))

## Register the train and test datasets

In [ ]:
# the registration process is needed for detectron2 to be able to find the datasets
dataset_catalog_name_root = func_dataset.__name__[5:]
for dataset_type in ["train", "test"]:
    dataset_catalog_name = f"{dataset_catalog_name_root}_{dataset_type}"
    DatasetCatalog.register(dataset_catalog_name, lambda dataset_type=dataset_type: func_dataset(folder=os.path.join(path_organized, dataset_type)))
    MetadataCatalog.get(dataset_catalog_name).set(thing_classes=["CFU"])


In [ ]:
dataset_gsk_train_metadata = MetadataCatalog.get(f"{dataset_catalog_name_root}_train")
dataset_gsk_test_metadata = MetadataCatalog.get(f"{dataset_catalog_name_root}_test")
dataset_gsk_train_metadata
dataset_gsk_test_metadata

## Recompute KPIs if new data and/or new KPIs on which to assess models

This ensures the models KPIs stay up to date so the ML API, which fetches the best model according to those metrics, makes educated choices

In [ ]:
# check if data folders or CFUEvaluator KPIs changed and if so, recompute the kpis on the updated test data and update the models_kpis
models_kpis = check_retest_models(models_kpis=models_kpis, data_folders=folders_positive + folders_negative, n_examples=count_files(path_organized))

# write the updated models kpis to disk and upload to blob
write_zip_upload_models_kpis(models_kpis=models_kpis)


## Visualize the data (optional)

In [ ]:
print("get dataset_dicts")
dataset_train = func_dataset(os.path.join(path_organized, "train"))
dataset_test = func_dataset(os.path.join(path_organized, "test"))

In [ ]:
display_enhanced_dataset(enhance_detectron2_dataset(dataset=[Detectron2DatasetElement(**dataset_el) for dataset_el in dataset_train]))


In [ ]:
display_enhanced_dataset(enhance_detectron2_dataset(dataset=[Detectron2DatasetElement(**dataset_el) for dataset_el in dataset_test]))

## Analyse data (optional)

In [ ]:
# compute data statistics
bbox_counts_train, bbox_sizes_train, bbox_aspects_train, pixels_blue_train, pixels_green_train, pixels_red_train = get_dataset_stats([Detectron2DatasetElement(**dataset_el) for dataset_el in dataset_train])
bbox_counts_test, bbox_sizes_test, bbox_aspects_test, pixels_blue_test, pixels_green_test, pixels_red_test = get_dataset_stats([Detectron2DatasetElement(**dataset_el) for dataset_el in dataset_test])

In [ ]:
# Display key datasets variables distributions
(
    display_violin(bbox_counts_train, name="Bbox counts Train")
    + display_violin(bbox_counts_test, name="Bbox counts Test")
)

(
    display_violin(bbox_sizes_train, name="Bbox sizes Train")
    + display_violin(bbox_sizes_test, name="Bbox sizes Test")
)

(
    display_violin(bbox_aspects_train, name="Bbox aspect ratios Train")
    + display_violin(bbox_aspects_test, name="Bbox aspect ratios Test")
)

sample_size = 500
(
    display_violin(np.random.choice(pixels_blue_train, size=sample_size, replace=False), name="Images blue channel values Train")
    + display_violin(np.random.choice(pixels_green_train, size=sample_size, replace=False), name="Images green channel values Train")
    + display_violin(np.random.choice(pixels_red_train, size=sample_size, replace=False), name="Images red channel values Train")
    + display_violin(np.random.choice(pixels_blue_test, size=sample_size, replace=False), name="Images blue channel values Test")
    + display_violin(np.random.choice(pixels_green_test, size=sample_size, replace=False), name="Images green channel values Test")
    + display_violin(np.random.choice(pixels_red_test, size=sample_size, replace=False), name="Images red channel values Test")
).cols(3)

# 2. Train Instance Segmentation model (DL, aka Deep Learning)


In [ ]:
# display trained model names
print(model_names)

## Load and modify the training config

In [ ]:
from_detectron2_model_zoo = True
backbone = Backbone.R50_FPN
device = Device.GPU
existing_model_name = "c2cc9388-7ef8-4ac9-af0a-4eeda21ec99c"
iterations = 36_000


if from_detectron2_model_zoo:
    cfg = prepare_cfu_detectron2_config_from_detectron2_model_zoo(backbone=backbone, device=device, iterations=iterations)

else:
    cfg = prepeare_cfu_detectron_config_from_existing_model(existing_model_name=existing_model_name, iterations=iterations)

print(cfg)


## Instantiate trainer

In [ ]:
trainer = CFUTrainer(cfg)

trainer.resume_or_load(resume=True)
if cfg.TEST.AUG.ENABLED:
    trainer.register_hooks(
        [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
    )

## Visualize Augmentations (optional)

as generated by the train and test data batches

In [ ]:
train_data_loader = trainer.build_train_loader(cfg)
data_iter = iter(train_data_loader)

In [ ]:
batchs = [next(data_iter) for _ in range(100)]  # To get a bigger batch to analyse
batch = []
for _batch in batchs:
    batch.extend(_batch)

display_batch(batch, cfg=cfg)

In [ ]:
test_data_loader = trainer.build_test_loader(cfg, dataset_name=dataset_gsk_test_metadata.name)
data_iter = iter(test_data_loader)


In [ ]:
batchs = [next(data_iter) for _ in range(10)]  # To get a bigger batch to analyse
batch = []
for _batch in batchs:
    batch.extend(_batch)

display_batch(batch, cfg=cfg)

## Start Training

In [ ]:
trainer.train()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, MODEL_WEIGHTS_NAME)


## Assess trained model

In [ ]:
# Construct model info (name, datetime, ..., KPIs)
model_info = construct_model_info(cfg=cfg)


## Save temporary config

In [ ]:
# write the config next to the final model and events
model_config_path = os.path.join(cfg.OUTPUT_DIR, MODEL_CONFIG_NAME)
logger.info(f"Writing the model config to: {model_config_path}.")
with open(model_config_path, "w") as f:
    f.write(cfg.dump())

# 3. Train Quantiles model (LightGBM)

## Generate lightGBM augmented train and test datasets

- Compute and gather the scores on augmented training and testing data
- Convert the augmented training and testing scores into X_train and X_test
- Train quantile regression models to estimate the 95% prediction interval of the DL model predictions
- Assess the quantile regression models
  - Calibration - How close to 95% do they cover the actual counts
  - Sharpness - How sharp are the intervals (the smaller the MSE/MAE, ..., the better)
- Persist the models on local disk

In [ ]:

n_bins = 100
n_exples_train = 10_000
n_exples_test = 1_000

X_train, y_train = generate_pi_dataset(model_name=model_info.name, data_type=DataType.TRAIN, n_examples=n_exples_train, n_bins=n_bins, augmentations=CFUTrainer.augmentations_test)
X_test, y_test = generate_pi_dataset(model_name=model_info.name, data_type=DataType.TEST, n_examples=n_exples_test, dataset_name=dataset_gsk_test_metadata.name, n_bins=n_bins, augmentations=CFUTrainer.augmentations_test)


## Start training

In [ ]:
n_hp_points = 32_768
hp_space_size = HPSpace.WIDE
quantiles = [0.025, 0.975]

EPS = 1e-10

regs = [
    ModelQuantile(
        model=train_quantile_regressor(X_train=X_train, y_train=y_train, quantile=quantile, n_hp_points=n_hp_points, hp_space_size=hp_space_size, train_size=0.7, cv=3),
        name=f"Quantile model {quantile * 100:.1f}th percentile",
        log_y=False,
        eps=EPS,
        n_bins=100,
        column_names=list(X_train.columns),
        quantile=Quantile(expected=quantile)
    )
    for quantile in quantiles
]

In [ ]:
# update actual quantiles
y_train_pred_low = regs[0].predict(X_train)
y_test_pred_low = regs[0].predict(X_test)
y_train_pred_high = regs[1].predict(X_train)
y_test_pred_high = regs[1].predict(X_test)

regs[0].quantile.actual_train = (y_train < y_train_pred_low).sum() / len(y_train)
regs[0].quantile.actual_test = (y_test < y_test_pred_low).sum() / len(y_test)

regs[1].quantile.actual_train = (y_train < y_train_pred_high).sum() / len(y_train)
regs[1].quantile.actual_test = (y_test < y_test_pred_high).sum() / len(y_test)

expected_interval = quantiles[1] - quantiles[0]
actual_interval_train = regs[1].quantile.actual_train - regs[0].quantile.actual_train
actual_interval_test = regs[1].quantile.actual_test - regs[0].quantile.actual_test

model_quantiles_interval = ModelQuantilesInterval(
    low=regs[0],
    high=regs[1],
    expected_interval=expected_interval,
    actual_interval_train=actual_interval_train,
    actual_interval_test=actual_interval_test,
    calibration_train=calibration(pi_expected=expected_interval, pi_actual=actual_interval_train),
    calibration_test=calibration(pi_expected=expected_interval, pi_actual=actual_interval_test),
    sharpness_train=Sharpness(
        mae=mae(y_train_pred_high.to_numpy(), y_train_pred_low.to_numpy()),
        mse=mse(y_train_pred_high.to_numpy(), y_train_pred_low.to_numpy())
    ),
    sharpness_test=Sharpness(
        mae=mae(y_test_pred_high.to_numpy(), y_test_pred_low.to_numpy()),
        mse=mse(y_test_pred_high.to_numpy(), y_test_pred_low.to_numpy())
    ),
)


In [ ]:
# persist quantile models and data in the corresponding Instance Segmentation model folder
joblib.dump(model_quantiles_interval, filename=os.path.join(cfg.OUTPUT_DIR, MODELS_QUANTILES_NAME), compress=9, protocol=pickle.HIGHEST_PROTOCOL)

dataset_pi = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
}
joblib.dump(dataset_pi, filename=os.path.join(cfg.OUTPUT_DIR, DATASETS_QUANTILES_NAME), compress=9, protocol=pickle.HIGHEST_PROTOCOL)

# update the model_info with the quantiles
model_info.quantiles = [model_quantiles_interval.low.quantile, model_quantiles_interval.high.quantile]
model_info.quantiles_calibration_test = model_quantiles_interval.calibration_test
model_info.quantiles_sharpness_test = model_quantiles_interval.sharpness_test


# 4. Persist Instance Segmentation and Quantiles Models

### Update and upload the models KPIs

In [ ]:
# reload models_kpis in case another parallel training updated the available models in the meantime
models_kpis = download_load_models_kpis()

# Add the instance segmentation model info to model_kpis tracking
models_kpis.models.append(model_info)

# persist the updated model+kpis tracking
write_zip_upload_models_kpis(models_kpis=models_kpis)


### Clean the models folder and zip upload it

In [ ]:
# clean output dir and add cfg
clean_output_dir_add_cfg(cfg=cfg)

# zip upload that model folder and its content to blob
zip_upload_model_to_blob(cfg.OUTPUT_DIR)


# 5. Analyse Instance segmentation and Quantiles models predictions

## List models and load one


In [ ]:
[model.name for model in models_kpis.models]

In [ ]:
# predictor = CFUPredictor(get_best_model_name(models_kpis=models_kpis))  # load best model
# predictor = CFUPredictor(os.path.basename(cfg.OUTPUT_DIR))  # load the model just trained
predictor = CFUPredictor("5c352834-3af1-4018-bac5-cc5dad43e202")

## Display Instance Segmentation predictions on train and test sets (optional)

In [ ]:
dataset_train_enhanced = enhance_detectron2_dataset(dataset=[Detectron2DatasetElement(**_dataset) for _dataset in dataset_train], predictor=predictor)

In [ ]:
dataset_test_enhanced = enhance_detectron2_dataset(dataset=[Detectron2DatasetElement(**_dataset) for _dataset in dataset_test], predictor=predictor)

In [ ]:
display_enhanced_dataset(dataset_train_enhanced)

In [ ]:
display_enhanced_dataset(dataset_test_enhanced)

## Analyse best models (optional)

- model parameters
- feature importance
- PI
  - calibration
  - sharpness


In [ ]:
dataset_pi = joblib.load(os.path.join(PATH_MODELS_ON_DISK, predictor.model_name, DATASETS_QUANTILES_NAME))
regs = [predictor.models_pi.low, predictor.models_pi.high]

X_train = dataset_pi["X_train"]
y_train = dataset_pi["y_train"]
X_test = dataset_pi["X_test"]
y_test = dataset_pi["y_test"]
X_train.shape
y_train.shape


In [ ]:
# Models hyperparameters
for reg in regs:
    print(reg.name)
    print(reg.model.get_params())
    

In [ ]:
# Feature importance
display_features_importance(regs=regs, nlargest=20)


In [ ]:
# Verify expected and actual quantiles
verify_quantiles_predictions(y=y_train, y_preds_low=regs[0].predict(X_train), y_preds_high=regs[1].predict(X_train), quantile_low=regs[0].quantile.expected, quantile_high=regs[1].quantile.expected)
verify_quantiles_predictions(y=y_test, y_preds_low=regs[0].predict(X_test), y_preds_high=regs[1].predict(X_test), quantile_low=regs[0].quantile.expected, quantile_high=regs[1].quantile.expected)


In [ ]:
# Display True count, DL predictions and Quantiles PI
display_pi(X=X_train, y=y_train, y_preds_dl=X_train.iloc[:, 49], model_quantile_low=regs[0], model_quantile_high=regs[1])
display_pi(X=X_test, y=y_test, y_preds_dl=X_test.iloc[:, 49], model_quantile_low=regs[0], model_quantile_high=regs[1])

# 6. Compare DL and Scan4000 (optional)

In [ ]:
path_scan4000_reports = "./scan4000/reports"
path_pdfs = os.path.join(path_scan4000_reports, "PDF")
path_scan4000_extracted_original_images = "./scan4000_original"
Path(path_scan4000_extracted_original_images).mkdir(parents=True, exist_ok=True)
path_scan4000_extracted_labeled_images = "./scan4000_labeled"
Path(path_scan4000_extracted_labeled_images).mkdir(parents=True, exist_ok=True)
path_scan4000_extracted_predicted_images = "./scan4000_predicted"
Path(path_scan4000_extracted_predicted_images).mkdir(parents=True, exist_ok=True)
path_dl_predicted_images = "./dl_predicted"
Path(path_dl_predicted_images).mkdir(parents=True, exist_ok=True)

## Prepare Scan4000 pdfs and excel file

In [ ]:
# download scan4000 pdfs and extract metadata and original and scan4000 predicted images
# --------------------------------------------------------------------------------------

# download pdf reports and excel file
download_unzip_clean_zip(
    name="reports.zip",
    path_on_disk="./scan4000",
    path_on_blob=f"{ACCOUNT_URL}/{CONTAINER}/scan4000",
    overwrite=False,
)

In [ ]:
_reports = []
for pdf_name in tqdm(os.listdir(path_pdfs)):
    # extract original images
    _reports.append(extract_all_from_scan4000_report(path_pdf=os.path.join(path_pdfs, pdf_name), path_extracted_images=path_scan4000_extracted_original_images))
    # extract scan4000 predicted images
    _ = extract_all_from_scan4000_report(path_pdf=os.path.join(path_pdfs, pdf_name), path_extracted_images=path_scan4000_extracted_predicted_images, image_type="predicted")

reports = pd.concat(_reports, axis=0, ignore_index=True)
reports.shape
reports.head()

In [ ]:
# download excel file
scan4000_excel = load_scan4000_excel(filename=os.path.join(path_scan4000_reports,"Draft_POC_QCBE_DATA_Tanguy_WIP_V1.xlsx"))
scan4000_excel

In [ ]:
# merge the excel data (id = Sample N° in PDF + pdf_name) with the pdf metadata and images (id = Sample N° + pdf_name)
reports_counts = pd.merge(reports, scan4000_excel,  how="left", left_on=["Sample N°", "pdf"], right_on=["Sample N° in PDF", "pdf"])
reports_counts.shape
reports_counts.head()

## Compute DL predictions on matching test images

In [ ]:
matching_test_images = MATCHES

dataset_detectron2 = joblib.load(os.path.join(path_organized, "test", DATASET_DETECTRON2_NAME))
_matches_scan4000_dl = []
for dl_image, report_image, is_match in tqdm(matching_test_images):
    if is_match is True:
        path_dl_image_original = os.path.join(path_organized, "test", dl_image)
        path_dl_image_predicted = os.path.join(path_dl_predicted_images, dl_image)
        path_scan4000_original_image = os.path.join(path_scan4000_extracted_original_images, report_image)  
        path_scan4000_predicted_image = os.path.join(path_scan4000_extracted_predicted_images, report_image)
        path_image_labeled = os.path.join(path_scan4000_extracted_labeled_images, report_image)
        
        count_scan4000 = int(reports_counts["Count"][reports_counts["image"] == os.path.join(path_scan4000_extracted_original_images, report_image)])
        dataset_detectron2_el = [_dataset_detectron2_el for _dataset_detectron2_el in dataset_detectron2 if dl_image in _dataset_detectron2_el.file_name][0]
        count_true = len(dataset_detectron2_el.annotations)
        
        # compute dl predictions and predicted image
        image_original = np.array(Image.open(path_dl_image_original))[:, :, :3]
        predictions_dl = predictor.predict_quantiles([image_original[:, :, ::-1]])[0]
        count_dl = predictions_dl.model_prediction.prediction
        pi_low = predictions_dl.quantiles_predictions[0].prediction
        pi_high = predictions_dl.quantiles_predictions[1].prediction
        
        # generate predicted image
        v = Visualizer(image_original)
        out = v.draw_instance_predictions(predictions_dl.instances)
        image_dl = out.get_image()
        _ = cv2.imwrite(path_dl_image_predicted, image_dl[:, :, ::-1])
        
        # generate labelled image
        image_labeled = (
            Visualizer(image_original).draw_dataset_dict(dataset_detectron2_el.dict()).get_image()
        )
        _ = cv2.imwrite(path_image_labeled, image_labeled[:, :, ::-1])
        
        # construct results
        _matches_scan4000_dl.append(
            (
                path_dl_image_original,
                path_scan4000_original_image,
                path_image_labeled,
                path_dl_image_predicted,
                path_scan4000_predicted_image,
                count_dl,
                count_scan4000,
                count_true,
                pi_low,
                pi_high
            )
        )
matches_scan4000_dl = pd.DataFrame(_matches_scan4000_dl, columns=["dl image original", "scan4000 image original", "image labeled", "dl image predicted", "scan4000 image predicted", "dl count", "scan4000 count", "true count", "PI low", "PI high"])

matches_scan4000_dl

## Compare Scan4000 and DL KPIs

In [ ]:
# compare scan4000 and DL KPIs
cfu_evaluator = CFUEvaluator()
cfu_evaluator.gt_counts = matches_scan4000_dl["true count"].to_list()
cfu_evaluator.pred_counts = matches_scan4000_dl["dl count"].to_list()
kpis_dl = cfu_evaluator.evaluate()



def show_ratios(ratio_name, n_examples: int) -> None:
    ratio = kpis_dl["ratio"][ratio_name]
    n_local_examples = int((ratio / 100) * n_examples)
    print(f"ratio {ratio_name} | {ratio:.1f} % | {n_local_examples} examples")

n_examples = len(cfu_evaluator.gt_counts)
show_ratios("0", n_examples=n_examples)
show_ratios("1-5", n_examples=n_examples)
show_ratios("6-10", n_examples=n_examples)
show_ratios("11+", n_examples=n_examples)

kpis_dl["main"]

cfu_evaluator.pred_counts = matches_scan4000_dl["scan4000 count"].to_list()
kpis_scan4000 = cfu_evaluator.evaluate()
kpis_scan4000["main"]




In [ ]:

# 5c352834-3af1-4018-bac5-cc5dad43e202
{
    'MAE (counts) 0': 0.0,
    'MAE (counts) 1-5': 0.25,
    'MAE (counts) 6-10': 1.1,
    'MAPE (% counts) 11+': 6.7
}


# 9a6b6aac-17d0-4bb3-87f3-2419f60fb2ba
{'MAE (counts) 0': 0.0,
 'MAE (counts) 1-5': 0.2,
 'MAE (counts) 6-10': 0.9,
 'MAPE (% counts) 11+': 7.9}

# Scan4000
{
    'MAE (counts) 0': 0.06,
    'MAE (counts) 1-5': 21.5,
    'MAE (counts) 6-10': 29.9,
    'MAPE (% counts) 11+': 169.2,
}

In [ ]:
# compare scan4000 and DL images
compare_scan4000_and_dl_images(matches_scan4000_dl, width=800, height=800)
   
    

In [ ]:
path_predictions = os.path.join(PATH_PREDICTIONS_ON_DISK, predictor.model_name)
save_examples(matches=matches_scan4000_dl, output_folder=path_predictions)

In [ ]:
zip_upload_predictions_to_blob(predictions=path_predictions, overwrite=True, compress=False)